In [11]:
import re

def label_node(ckt_graph,input_vars,key):
    if key in input_vars:
        ckt_graph[key]["level"] = 0
        return 1
    elif ckt_graph[key]["level"] != -1:
        return ckt_graph[key]["level"]+1
    else:
        max_level = -1 
        for input in ckt_graph[key]["inputs"]:
            temp_level = label_node(ckt_graph,input_vars,input)
            if temp_level>max_level:
                max_level = temp_level
        ckt_graph[key]["level"] = max_level
        return max_level+1

def bench2graph(benchfile):
    with open(benchfile, 'r') as file:
        lines = file.readlines()
    
    ckt_graph={}
    output_vars=[]
    input_vars=[]
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)
            if var_name in ckt_graph.keys():
                ckt_graph[var_name]["gate"] = gate_match[1]
                ckt_graph[var_name]["inputs"] = gate_match[2:]
            else:  
                node={}  
                node["level"] = -1
                node["outbound"] = []
                node["gate"] = gate_match[1]
                node["inputs"] = gate_match[2:]
                ckt_graph[var_name] = node
            for inp_node in gate_match[2:]:
                if inp_node in ckt_graph.keys():
                    ckt_graph[inp_node]["outbound"].append(var_name)
                else:
                    node={}
                    node["level"] = -1
                    node["outbound"] = [var_name]
                    node["gate"] = ""
                    node["inputs"] = []
                    ckt_graph[inp_node] = node
                    
        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    output_vars.append(match.group(1))
    
    for key in ckt_graph.keys():
        if key in output_vars:
            label_node(ckt_graph,input_vars,key)

    return ckt_graph,input_vars,output_vars,wires


def graph2bench(ckt_node,inputs,outputs,bench_name,input_node =False):
    bench=[f"# {bench_name}",f"# {len(inputs)} INPUTS", f"# {len(outputs)} OUTPUTS", f"# {len(ckt_node.keys())} Gates", "\n"]

    for input in inputs:
        bench.append(f"INPUT({input})")
    for output in outputs:
        bench.append(f"OUTPUT({output})")
    bench.append("\n")

    gate_no = 0
    key_len = len(ckt_node.keys())
    if input_node:
        key_len -= len(inputs)
    appending_level = 1
    while gate_no<key_len:
        for key in ckt_node.keys():
            if ckt_node[key]["level"]==appending_level:
                if len(ckt_node[key]["inputs"])>1:
                    gate_inp = ", ".join(ckt_node[key]["inputs"])
                else: gate_inp = ckt_node[key]["inputs"][0]
                gate_name = ckt_node[key]["gate"]
                bench.append(f"{key} = {gate_name}({gate_inp})")
                gate_no += 1
        appending_level += 1

    with open(bench_name+".bench", 'w') as file:
        file.write("\n".join(bench))
        print(bench_name+" bench file created")
                

In [12]:

def convert_gate_line(line, gate_count):
    match = re.findall(r'\b\w+\b', line)
    gate_dict=[ "AND", "NAND", "OR", "NOR", "XOR", "XNOR", "NOT", "BUF", "DFF"]
    
    if match:
        output_var = match[0]
        gate_type = match[1]  # Convert gate type to lowercase
        inputs = match[2:]
        
        if gate_type in gate_dict:
            gate_unique_name = f"{gate_type}{len(inputs)}_{gate_count}"
            converted_line = f"{gate_type.lower()} {gate_unique_name} ({output_var}, {', '.join(inputs)});"
            return gate_type, converted_line
    
    return None, None


def bench2verilog(file_path, file_name):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gate_count = 1
    input_vars = []
    output_vars = []
    assigned_vars = []
    dff_module = [
            "module DFF(Q, clk, D);",
            "input D;",
            "input clk;",
            "output Q;",
            "always @(clk)", 
            "begin",
            "  Q <= D;", 
            "end", 
            "endmodule"]
    opr_lines=[]
    gate_types=[]

    # Regular expressions to match INPUT, OUTPUT, and gate assignments
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    assign_pattern = re.compile(r'^(\w+)\s*=\s*(NAND|NOR|AND|OR|XOR|XNOR|NOT|DFF|BUF)\((.*)\)')

    for line in lines:
        line = line.strip()
        if line.startswith("INPUT"):
            match = input_pattern.match(line)
            if match:
                input_vars.append(match.group(1))
        elif line.startswith("OUTPUT"):
            match = output_pattern.match(line)
            if match:
                output_vars.append(match.group(1))
        else:
            match = assign_pattern.match(line)
            if match:
                assigned_vars.append(match.group(1))
                gate_type,opr_line = convert_gate_line(line,gate_count)
                if opr_line is not None:
                    opr_lines.append(opr_line)
                    if gate_type not in gate_types:
                        gate_types.append(gate_type) 
                    gate_count += 1

    # Intermediate variables are those assigned but not inputs or outputs
    intermediate_vars = [var for var in assigned_vars if var not in input_vars and var not in output_vars]

    module_heading = f"module {file_name}({','.join(input_vars + output_vars)});"
    input_line = f"input {','.join(input_vars)};"
    output_line = f"output {','.join(output_vars)};"
    wire_line = f"wire {','.join(intermediate_vars)};"
    termination_line = "endmodule"
    main_module="\n".join([module_heading,"",input_line,output_line,wire_line,""]+opr_lines+["",termination_line])
    
    if "DFF" in gate_types:
        main_module = "\n".join(dff_module + ["\n", main_module])

    with open(file_name+".v", 'w') as file:
        file.write(main_module)
        print(file_name+" verilog file created")


In [28]:
def build_cone_backward(cone_graph,ckt_graph,inputs,key):
    for input in ckt_graph[key]["inputs"]:
        cone_graph[key] = ckt_graph [key]
        if input not in cone_graph.keys():
            build_cone_backward(cone_graph,ckt_graph,inputs,input) 
    return

def build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,key,lvl):
    if key in outputs:
        cone_graph[key] = ckt_graph [key]
        cone_output.append(key)
        return
    else:
        for outbound in ckt_graph[key]["outbound"]:
            if outbound not in cone_graph.keys():
                cone_graph[outbound] = ckt_graph [outbound]
                build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,outbound,lvl+1)
        if lvl<4:
            for input in ckt_graph[key]["inputs"]:
                if input not in cone_graph.keys():
                    build_cone_backward(cone_graph,ckt_graph,inputs,input)
            
        return


def logic_cone_forward(benchfile,target_text,cone_name):
    ckt_graph,inputs,outputs,wires=bench2graph(benchfile)
    cone_graph={}
    cone_input=[]
    cone_output=[]
    for input in inputs:
        if target_text in input:
            cone_input.append(input)
            build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,input,1)
    

    cone_wires = list(cone_graph.keys())
    inpx=[]
    wirex=[]
    for key in cone_graph.keys():
        new_inp=[]
        for input in cone_graph[key]["inputs"]:
            if target_text not in input:
                if input in inputs:
                    if input in inpx:
                        new_inp.append(f"INPUTx{inpx.index(input)}")
                    else:
                        new_inp.append(f"INPUTx{len(inpx)}")
                        inpx.append(input)
                elif input in cone_wires:
                    new_inp.append(input)
                elif input in wires:
                    if input in wirex:
                        new_inp.append(f"WIREx{wirex.index(input)}")
                    else:
                        new_inp.append(f"WIREx{len(wirex)}")
                        wirex.append(input)
                else:
                    print("undefined input") 
                    new_inp.append(input)
            else: new_inp.append(input)
        cone_graph[key]["inputs"] = new_inp


        
    graph2bench(cone_graph,cone_input,cone_output,cone_name)




In [30]:
logic_cone_forward("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c6288/c6288_ttlock_32k.bench","keyinput","ttlock")

ttlock bench file created


In [17]:
logic_cone_forward("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/bench_ckt/c17.bench","gatx","c17")

c17 bench file created


In [16]:
bench2verilog("/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/src/cac.bench","cac")

FileNotFoundError: [Errno 2] No such file or directory: '/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/src/cac.bench'

In [32]:
#ttlock 32 bit
import random
org_file = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432.bench"
lock_file = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/obfuscated/ttlock.bench"
file_name = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_ttlock_32k.bench"

def bench2list(org_file, add_out=True):
    with open(org_file, 'r') as file:
            lines = file.readlines()
        
    output_vars=[]
    input_vars=[]
    io_lines = []
    gate_lines = []
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_lines.append(line)
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)

        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
                    io_lines.append(line)
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    if add_out:
                        io_lines.append(line)
                    output_vars.append(match.group(1))
                    
    
    return gate_lines,io_lines,input_vars,output_vars,wires

org_gates, org_io, org_inputs, org_outputs, org_wires = bench2list(org_file)
lock_gates, lock_io, lock_inputs, lock_outputs, lock_wires = bench2list(lock_file, False)

bench = org_io+lock_io

gate_index = org_wires.index(org_outputs[-1])
org_gates[gate_index] = org_gates[gate_index].replace(org_outputs[-1],org_outputs[-1]+"_enc")
lock_gates[-1] = lock_gates[-1].replace(lock_outputs[0],org_outputs[-1])
lock_gates[-4] = lock_gates[-4].replace("WIREx0",org_outputs[-1]+"_enc")
lock_gates[-5] = lock_gates[-5].replace("WIREx0",org_outputs[-1]+"_enc")

wirex=[]
for i in range(len(lock_gates)):
    gate_match = re.findall(r'\b\w+\b', lock_gates[i])
    for inp in gate_match[2:]:
        if "WIREx" in inp:
            wire_index = int(inp.split("x")[-1].strip())
            if wire_index>=len(wirex):
                while (org_wires[wire_index] in org_outputs) and (org_wires[wire_index] in wirex):
                    wire_index = random.randint(0,len(org_wires)-1)
                wirex.append(org_wires[wire_index])
                lock_gates[i] = lock_gates[i].replace(inp,org_wires[wire_index]) 
            else:
                lock_gates[i] = lock_gates[i].replace(inp,wirex[wire_index])
        elif "INPUTx" in inp:
            inp_index = int(inp.split("x")[-1].strip())
            lock_gates[i] = lock_gates[i].replace(inp,org_inputs[inp_index])

bench += org_gates+lock_gates
with open(file_name, 'w') as file:
    file.write("\n".join(bench))
    print(file_name+" bench file created")

/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_ttlock_32k.bench verilog file created
